In [6]:
from PIL import Image, ImageFont, ImageDraw
import os
import math

In [7]:
def get_dynamic_font(dpi, height_in_inches=0.25, font_name="arial.ttf"):
    from matplotlib import font_manager
    font_path = font_manager.findfont(font_name)
    font_size_px = int(height_in_inches * dpi)
    try:
        font = ImageFont.truetype(font_path, font_size_px)
    except Exception as e:
        print("Failed to load truetype font:", e)
        font = ImageFont.load_default()
    return font

In [8]:
def combine_images(image_paths, output_path, columns=2, crop_box=None, index_h=0.25):
    labels = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)', '(i)', '(j)']
    if not image_paths:
        raise ValueError("No images provided.")
    if crop_box is None:
        print("No cropping is applied")
        crop_box = (0, 0, 0, 0)

    # Open all images
    images = []
    for p in image_paths:
        img = Image.open(p)
        print(img.size)
        cropped_img = img.crop(crop_box)
        images.append(cropped_img)

    # Ensure all images have the same DPI
    dpis = [img.info.get("dpi", (72, 72)) for img in images]
    if not all(dpi == dpis[0] for dpi in dpis):
        print("Warning: Images have different DPI. Using DPI of the first image.")

    dpi = dpis[0]

    # Get max width and height of all images
    widths, heights = zip(*(img.size for img in images))

    max_width = max(widths)
    max_height = max(heights)
    print(max_width, max_height)

    # Calculate rows
    rows = math.ceil(len(images) / columns)

    # Create new blank image
    combined_width = columns * max_width
    combined_height = rows * max_height

    combined_image = Image.new("RGB",
                               (combined_width, combined_height),
                               color=(255, 255, 255))

    # Get font
    font = get_dynamic_font(dpi[0], height_in_inches=index_h)

    # Paste images into the combined image
    for index, image in enumerate(images):
        row = index // columns
        col = index % columns
        x = col * max_width
        y = row * max_height
        combined_image.paste(image, (x, y))
        draw = ImageDraw.Draw(combined_image)
        draw.text((x+10, y+10), labels[index], font=font, fill=(0, 0, 0))

    # Save the result with DPI
    combined_image.save(output_path, dpi=dpi)
    print(f"Saved combined image to {output_path} with DPI {dpi}")

In [13]:
nCols = 2
indexH = 0.2
figsdir = '/glade/work/swei/projects/mmm.pace_aod/plots/stats/vs_AERONET'
# figsdir = '/glade/work/swei/projects/mmm.pace_aod/plots/gridded'
filetmp = '{figsdir}/{plotType}.{loopname}.{timeRange}.png'
# figsdir = '/glade/work/swei/projects/mmm.pace_aod/plots/cycles/obs'
# filetmp = '{figsdir}/{obsname}/{plotType}.{loopname}.{timeRange}.png'
savedir = '/glade/work/swei/projects/mmm.pace_aod/plots/combined'
plotType = 'hfx_vs_AERONET.hist2d'
sfc_list = [
    'land',
    'water',
]
obs_list = [
    # 'water', 'land'
]
set_dict = {
    'main': [
        'pace_aod',
        'modis_aqua_aod',
        # 'pace_aod-modis_aqua_aod',
        'viirs_aod_dt_n20',
        # 'pace_aod-viirs_aod_dt_n20',
        'viirs_aod_db_n20',
        # 'pace_aod-viirs_aod_db_n20',
    ],
    'sup': [
        'pace_aod',
        'modis_terra_aod',
        'viirs_aod_dt_npp',
        'viirs_aod_db_npp',
    ]
}
set_type = 'main'
# timeRange = '202411'
timeRange = '2024110100_2024113018'
loop_list = set_dict[set_type]
# cropbox = (left, top, right, bottom)
# cropbox = (400, 1000, 6000, 3800) # obs2d
# cropbox = (200, 200, 3800, 2100) # obs_mean
# cropbox = (0, 200, 3600, 2880) # allombhist
cropbox = (150, 100, 3450, 3450) # hist2d
# cropbox = (200, 220, 3820, 2000) # obs_count

In [14]:
image_paths = []
for loopname in loop_list:
    tmpfile = filetmp.format(figsdir=figsdir, loopname=loopname, plotType=plotType, timeRange=timeRange)
    if not os.path.exists(tmpfile):
        raise Exception(f'{tmpfile} does not exist')
    image_paths.append(tmpfile)
combine_images(image_paths, f'{savedir}/{plotType}.{set_type}.{timeRange}.png', columns=nCols, crop_box=cropbox, index_h=indexH)

(3529, 3529)
(3529, 3529)
(3529, 3529)
(3529, 3529)
3300 3350
Saved combined image to /glade/work/swei/projects/mmm.pace_aod/plots/combined/hfx_vs_AERONET.hist2d.main.2024110100_2024113018.png with DPI (599.9988, 599.9988)
